In [ ]:
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from scipy.io import readsav
import array

# Leggere tutto il contenuto del file .dat in una volta
with open('/content/ibis_config_20150518.dat', 'r') as file:
    content = file.read()

# Dividere il contenuto in righe
lines = content.splitlines()

date = lines[9] #data delle osservazioni dal config
hhmmss = lines[10] #time of the obs dal config
filtro = '6173' # decide later where this variable should be taken
print(filtro)
#directories where the final fits file for flat and dark will be placed for the flat file

#dir_output = filtro+'flat.sav'
#dir_output = filtro+'dark.sav'

#directory temporanea

dir_temp = '/content/'

#ff_file = glob.glob(dir_output+"_flat.fits")
#dd_file = glob.glob(dir_output+"_dark.fits")

# Load dark and flat file

dd_file = dir_temp+'6173_dark.sav'
ff_file = dir_temp+'6173_flat.sav'

dd = readsav(dd_file, verbose=True)
ff = readsav(ff_file, verbose=True)

#for flat and dark in fits format
#with fits.open(ff_files) as hdul:
#ff = hdul[1].data
#ff_h = hdul[1].header
#with fits.open(dd_files) as hdul:
#dd = hdul[1].data
#dd_h = hdul[1].header

#Get dimensions

dim = np.shape(ff.flat)
Nx = dim[1]
Ny = dim[2]
Npol = lines[98]
Npol = int(Npol[9:10])
#Npol = int(lines[98])
Nwave = int(dim[0]/Npol) #(lines[98])

dual = lines[100]
dual = int(dual[7:8])

single = lines[101]
single = int(single[9:10])
#Create mean dark, no wavelength dependency assumed

dark = np.mean(dd.dark)

#Order flats when in rabbit mode --> CHECK if this is needed
#index = np.linspace(0, Nwave, Nwave+1)
index = np.arange(Nwave)

ftmp = ff.flat.copy()

for i in range(Nwave):
  ftmp[i*Npol:(i+1)*Npol-1,:,:] = np.copy(ff.flat[index.astype(int)[i]*Npol:(index.astype(int)[i]+1)*Npol-1,:,:])

flat = ftmp.copy()
ftmp == 0

# Load aperture mask APS

file_aps = '/content/6173_aps.fits'

# Open the FITS file of the mask
hdul = fits.open(file_aps)

# Visualise FITS information
#hdul.info()

aps = hdul[0].data
idx = np.where(aps > 1)


# Distinguish Modes : Spectropolarimetric mode

Pol = lines[99]
Pol = int(Pol[6:7])

if Pol == 1:

  print('Blueshift calculation for spectropolarimetric case')

temp = np.zeros(shape = [Ny,Nx])
flat_tmp = np.zeros(shape = [Npol, Nwave, Ny, Nx])

for j in range(Npol):
  for i in range(Nwave):
    temp[idx] = ((flat[i*Npol+1,:,:]-dark))[idx]
    flat_tmp[j,i,:,:] = temp

plt.plot(flat_tmp[0,:,500,250],marker='o')
plt.plot(flat_tmp[0,:,125,250],marker='*')
plt.plot(flat_tmp[0,:,900,250],marker='s')

#Define the wavelength range to be used to compute the blueshift.
#For all the filters except the Fe I 6302 will be from the second spectral point to the last -1.
wrange = [2, Nwave-1]

#Let's to calculate the offset using the ibis_core_map function
#when the files will be in fits the info_flat_nb will not be callabe with ff.info_flat_nb
#offset = ibis_core_map(flat_temp[:,wrange[0]:wrange[1],:,:], wrange, npoints, aps, ff.info_flat_nb, pol=Pol, dual=Dual, single=Single, stokes=Stokes)



# Distinguish Modes : Spectroscopic mode

else:

  print('Blueshift calculation for spectroscopic case')



  temp = np.zeros(shape = [Ny,Nx])
  flat_tmp = np.zeros(shape = [Nwave, Ny, Nx])

  for i in range(Nwave):
      temp[idx] = ((flat[i,:,:]-dark))[idx]
      flat_tmp[i,:,:] = temp

plt.plot(flat_tmp[0,:,500,500],marker='o')
plt.plot(flat_tmp[0,:,125,500],marker='*')
plt.plot(flat_tmp[0,:,900,500],marker='s')

#Define the wavelength range to be used to compute the blueshift.
#For all the filters except the Fe I 6302 will be from the second spectral point to the last -1.
wrange = [2, Nwave-1]

#Let's to calculate the offset using the ibis_core_map function
#when the files will be in fits the info_flat_nb will not be callabe with ff.info_flat_nb
offset = ibis_core_map(flat_temp[:,wrange[0]:wrange[1],:,:], wrange, npoints, aps, ff.info_flat_nb, pol=Pol, dual=Dual, single=Single, stokes=Npol)


#Save data


hdu1 = fits.PrimaryHDU(offset)
hdu2 = fits.ImageHDU(aps)
hdu3 = fits.ImageHDU(header)

# Modifica l'header con delle informazioni personalizzate
#hdu3.header['CREATOR'] = 'My script'
#hdu3.header['DATE'] = '2025-02-13'

# Salva il file FITS con l'header personalizzato
hdul = fits.HDUList([hdu1, hdu2, hdu3])
hdul.writeto(filtro+'blueshift.fits', overwrite=True)

print('Blueshift calculation done')
print('File saved')




IDL Save file is compressed
 -> expanding to /tmp/tmpi30t_fpm.sav
--------------------------------------------------
Date: Fri Dec 17 08:58:01 2021
User: root
Host: Sun18
--------------------------------------------------
Format: 12
Architecture: x86_64
Operating System: linux
IDL Version: 8.7.0
--------------------------------------------------
Successfully read 7 records of which:
 - 4 are of type VARIABLE
 - 1 are of type TIMESTAMP
 - 1 are of type VERSION
--------------------------------------------------
Available variables:
 - dark [<class 'numpy.ndarray'>]
 - dark_level [<class 'numpy.ndarray'>]
 - dark_rms [<class 'numpy.ndarray'>]
 - info_dark_nb [<class 'numpy.recarray'>]
--------------------------------------------------
IDL Save file is compressed
 -> expanding to /tmp/tmpth2rm_7x.sav
--------------------------------------------------
Date: Fri Dec 17 09:04:37 2021
User: root
Host: Sun18
--------------------------------------------------
Format: 12
Architecture: x86_64
Oper

In [ ]:

# Leggere tutto il contenuto del file .dat in una volta
with open('/content/ibis_config_20150518.dat', 'r') as file:
    content = file.read()

# Dividere il contenuto in righe
lines = content.splitlines()


In [ ]:
single = lines[98]
print(single)

stokes = 6


In [ ]:
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from scipy.io import readsav

def ibis_wavelength_shift(config_file, dark_file, flat_file, aps_file, filter, output_file, Pol=None, Dual=None, Single=None, Stokes=None):
    """
    Module that calcolate the blueshift.

    Args:
    - config_file: path of the config file (.dat) --> SOLVE LATER
    - dark_file: path of the dark
    - flat_file: path of the flat
    - aps_file: path of the mask to identify left and rigth channel
    - filter: filter
    - output_file: path of the output directory
    - Pol, Dual, Single, Stokes: Opzional parameters to understand how to pass (if applicable)

    Returns:
    - None, save the result in a FITS file.
    """

    # Read a config file

    with open(config_file, 'r') as file:
        content = file.read()

    # Divide the file in lines that can be estracted
    lines = content.splitlines()

    # Date, time and other information to be extracted
    date = lines[9]  # Observation Date
    hhmmss = lines[10]  # Observation time
    filtro = '6173'

    # Load the dark and flat files in .sav format
    dd = readsav(dark_file, verbose=True)
    ff = readsav(flat_file, verbose=True)

    # Get the dimension
    dim = np.shape(ff.flat)
    Nx = dim[1]
    Ny = dim[2]
    Npol = int(lines[98][9:10])
    Nwave = int(dim[0] / Npol)

    # Create mean dark, no wavelength dependency is assumed
    dark = np.mean(dd.dark)

    # Order flats when in a rabbit mode ---> CHECK if can be removed
    index = np.arange(Nwave)
    ftmp = ff.flat.copy()

    for i in range(Nwave):
        ftmp[i * Npol:(i + 1) * Npol, :, :] = np.copy(ff.flat[index.astype(int)[i] * Npol:(index.astype(int)[i] + 1) * Npol, :, :])

    flat = ftmp.copy()

    # Load the mask (APS) as a FITS file
    hdul = fits.open(aps_file)
    aps = hdul[0].data
    idx = np.where(aps > 1)

    # Distinguish modes : Spectropolarimetric mode

    if Pol == 1:
        print('Blueshift calculation for Spectropolarimetric case')

        temp = np.zeros(shape=[Ny, Nx])
        flat_tmp = np.zeros(shape=[Npol, Nwave, Ny, Nx])

        for j in range(Npol):
            for i in range(Nwave):
                temp[idx] = ((flat[i * Npol + 1, :, :] - dark))[idx]
                flat_tmp[j, i, :, :] = temp

        plt.plot(flat_tmp[0, :, 500, 250], marker='o')
        plt.plot(flat_tmp[0, :, 125, 250], marker='*')
        plt.plot(flat_tmp[0, :, 900, 250], marker='s')

    # Distinguish modes : Spectroscopic mode

    else:
        print('Blueshift calculation for Spectroscopic case ')

        temp = np.zeros(shape=[Ny, Nx])
        flat_tmp = np.zeros(shape=[Nwave, Ny, Nx])

        for i in range(Nwave):
            temp[idx] = ((flat[i, :, :] - dark))[idx]
            flat_tmp[i, :, :] = temp

        plt.plot(flat_tmp[0, :, 500, 500], marker='o')
        plt.plot(flat_tmp[0, :, 125, 500], marker='*')
        plt.plot(flat_tmp[0, :, 900, 500], marker='s')

    # Define the wavelength range
    wrange = [2, Nwave - 1]

    # Calculate the offset calling the ibis_core_map function
    offset = ibis_core_map(flat_tmp[:, wrange[0]:wrange[1], :, :], wrange, npoints, aps, ff.info_flat_nb, pol=Pol, dual=Dual, single=Single, stokes=Stokes)

    # Save results in a FITS file
    hdu1 = fits.PrimaryHDU(offset)
    hdu2 = fits.ImageHDU(aps)
    hdu3 = fits.ImageHDU(hdul[0].header)

    hdul_out = fits.HDUList([hdu1, hdu2, hdu3])
    hdul_out.writeto(output_file, overwrite=True)

    print('Blueshift calculation done')
    print('File saved in', output_file)

